In [1]:
!pip install aiogram
!pip install nest_asyncio

Это небольшой телеграм-бот, соданный чтобы помочь молодым людям с выбором ВУЗа на основании их профиля, города поступления, а также их баллов за ЕГЭ.

Университеты представлены в виде вложенных словарей. Если развивать проект, то можно добавить парсер сайтов, на которых собраны ВУЗы или добавить базу данных для хранения информации о ВУЗах.

In [2]:
from aiogram import Bot, Dispatcher, executor
from aiogram.types import *

# nest_asyncio обеспечивает работу aiogram в Jupiter Notebook,
# иначе интерпретатор выдает RuntimeError
import nest_asyncio

nest_asyncio.apply()

API_TOKEN = "5558759642:AAEBYYZzgJv4GPT8YMzf6qgAh14klSKowtY"

bot = Bot(token=API_TOKEN)
dispatcher = Dispatcher(bot)

# словарь из вариантов университетов в соответствии с профилем, городом и рангом (подробнее о ранговой системе ниже)
universitys: dict = {
    "Tехнологический профиль":
        {
            "Москва": {
                "1": "Российский университет транспорта\nhttps://www.miit.ru",
                "2": "Московский политехнический университет\nhttps://mospolytech.ru",
                "3": "Российский новый университет\nhttps://rosnou.ru",
                "4": "Московский государственный университет имени М.В. Ломоносова\nhttps://www.msu.ru"
            },
            "Санкт-Петербург": {
                "1": "Государственный университет флота имени адмирала С.О. Макарова\nhttps://gumrf.ru ",
                "2": "Санкт-Петербургский государственный морской технический университет\nhttps://www.smtu.ru",
                "3": "Санкт-Петербургский Гуманитарный университет профсоюзов\nhttps://www.gup.ru",
                "4": "СПБГУ - Санкт-Петербургский государственный университет\nhttps://spbu.ru"
            },
            "Екатеринбург": {
                "1": "Уральский государственный университет путей сообщения\nhttps://www.usurt.ru",
                "2": "Уральский федеральный университет имени Б.Н. Ельцина\nhttps://urfu.ru/ru/",
                "3": "Уральский государственный лесотехнический университет\nusfeu.ru",
                "4": "Уральский институт связи и информатики\nhttps://www.uisi.ru"
            },
            "Оренбург": {
                "1": "Оренбургский государственный университет\nhttp://www.osu.ru",
                "2": "Оренбургский институт путей сообщения\nhttps://www.samgups.ru",
                "3": "Оренбургский государственный педагогический университет\nospu.ru",
                "4": "Оренбургский филиал Поволжского университета телекоммуникаций и информатики\nhttps://of.psuti.ru"
            }
        },
    "Гуманитарный профиль": {
        "Москва": {
            "1": "Московский информационно-технологический университет\nhttps://masi.ru",
            "2": "Государственный университет по землеустройству\nhttps://www.guz.ru",
            "3": "Московский государственный гуманитарно-экономический университет\nhttps://mggeu.ru",
            "4": "Всероссийская академия внешней торговли\nhttps://www.vavt.ru"
        },
        "Санкт-Петербург": {
            "1": "Петербургский государственный университет Императора Александра I\nhttps://www.pgups.ru",
            "2": "Северо-Западный институт управления\nhttps://spb.ranepa.ru",
            "3": "Институт правоведения и предпринимательства\nhttps://www.иппспб.рф",
            "4": "Санкт-Петербургский Гуманитарный университет профсоюзов\nhttps://www.gup.ru"},
        "Екатеринбург": {
            "1": "Уральский федеральный университет имени Б.Н. Ельцина\nhttps://urfu.ru/ru/",
            "2": "Институт международных связей\nhttps://ims-ural.ru",
            "3": "Уральский институт управления\nhttps://ui.ranepa.ru",
            "4": "Уральский государственный юридический университет имени В.Ф. Яковлева\nusla.ru"
        },
        "Оренбург": {
            "1": "Оренбургский государственный аграрный университет\nhttps://orensau.ru",
            "2": "Оренбургский государственный педагогический университет\nospu.ru",
            "3": "Оренбургский институт имени О.Е. Кутафина (МГЮА)\nhttp://www.oimsla.edu.ru",
            "4": "Оренбургский государственный институт искусств им. Л. и М. Ростроповичей\nhttps://osiart.ru"
        }
        },
    "Естественно-научный профиль": {
        "Москва": {
            "1": "Российский государственный аграрный университет имени К.А. Тимирязева\nhttps://www.timacad.ru",
            "2": "МИРЭА – Российский технологический университет\nhttps://www.mirea.ru",
            "3": "Российский биотехнологический университет\nhttps://international.mgupp.ru",
            "4": "МГУ - Московский государственный университет имени М.В. Ломоносова\nhttps://www.msu.ru"},
        "Санкт-Петербург": {
            "1": "Санкт-Петербургский государственный лесотехнический университет\nhttps://spbftu.ru/public/",
            "2": "Северо-Западный государственный медицинский университет имени И.И. Мечникова\nhttps://szgmu.ru",
            "3": "Национальный исследовательский университет ИТМО\nitmo.ru",
            "4": "Институт медицинского образования имени В. А. Алмазова\nhttp://education.almazovcentre.ru"
        },
        "Екатеринбург": {
            "1": "Уральский федеральный университет имени Б.Н. Ельцина\nhttps://urfu.ru/ru/",
            "2": "Уральский государственный медицинский университет\nusma.ru",
            "3": "Уральский государственный лесотехнический университетn\nusfeu.ru",
            "4": "Уральский государственный медицинский университет\nusma.ru"
        },
        "Оренбург": {
            "1": "Оренбургский государственный аграрный университет\norensau.ru",
            "2": "Оренбургский государственный университет\nospu.ru",
            "3": "Оренбургский государственный медицинский университет\n",
            "4": "orgma.ru\n"
        }
        }
}

CITIES = ["Москва", "Санкт-Петербург", "Оренбург", "Екатеринбург"]


@dispatcher.message_handler(commands=["start"])
async def start(message):
    tiles_keyboard = ReplyKeyboardMarkup(resize_keyboard=True)
    start_button = KeyboardButton("НАЧНЕМ!")
    tiles_keyboard.add(start_button)

    await bot.send_message(message.chat.id, """Привет! Я чат-бот и могу помочь тебе определиться с выбором ВУЗа. 
Пока что я могу работать только по нескольким городам.""", reply_markup=tiles_keyboard)


@dispatcher.message_handler(lambda message: message.text == "НАЧНЕМ!")
async def select_profile(message):
    # здесь пользователь выбирает профиль, по которому хочет поступать с помощью кнопок
    tiles_keyboard = ReplyKeyboardMarkup(resize_keyboard=True)

    tiles_keyboard.add("Tехнологический профиль")
    tiles_keyboard.add("Гуманитарный профиль")
    tiles_keyboard.add("Естественно-научный профиль")

    await bot.send_message(message.chat.id, "Давай определимся с твоим профилем", reply_markup=tiles_keyboard)

    # следующие 3 хендлера с функциями обрабатывают нажатие соответствующих конопок профилей
    @dispatcher.message_handler(lambda message: message.text == "Tехнологический профиль")
    async def select_tech(message):
        await processing_exam_res(message, "Tехнологический профиль")

    @dispatcher.message_handler(lambda message: message.text == "Гуманитарный профиль")
    async def select_gum(message):
        await processing_exam_res(message, "Гуманитарный профиль")

    @dispatcher.message_handler(lambda message: message.text == "Естественно-научный профиль")
    async def select_natur(message):
        await processing_exam_res(message, "Естественно-научный профиль")


# большая функция обрабатывает главным образом результаты экзаменов
async def processing_exam_res(message, profile):
    tiles_keyboard = ReplyKeyboardMarkup(resize_keyboard=True)
    tiles_keyboard.add("НАЗАД")
    await bot.send_message(message.chat.id, f"Отлично! Выбран {profile}\nТеперь введи количество предметов (от 2 до 9)",
                           reply_markup=tiles_keyboard)

    # эта функция с хендлером отправляют пользователя назад к выбору профиля в случае, если он решит изменить данные
    @dispatcher.message_handler(lambda message: message.text == "НАЗАД")
    async def go_back(message):
        await select_profile(message)

    @dispatcher.message_handler(lambda message: message.text.isnumeric() and 2 <= int(message.text) <= 10)
    async def walk_on(message):
        global count_of_exams

        count_of_exams = await read_count_of_exams(message)

        await bot.send_message(message.chat.id, "Теперь введи сумму своих баллов (от 11 до 400)",
                               reply_markup=tiles_keyboard)

    @dispatcher.message_handler(lambda message: message.text.isnumeric() and 11 <= int(message.text) <= 450)
    async def walk_on1(message):
        # используем глобальную переменную для оперирования результатами экзаменов и их количество
        # не лучший шаг, но лучше, чем городить костыли
        global result_of_exam

        result_of_exam = await read_result_of_exam(message)

        tiles_keyboard = ReplyKeyboardMarkup(resize_keyboard=True)

        tiles_keyboard.add("Все верно. Продолжить")
        tiles_keyboard.add("Неверно. Вести данные заново")

        try:
            await bot.send_message(message.chat.id,
                                   f"""Итак, давай сверим данные\nколичесвто предметов: {count_of_exams}\n
общее количество баллов: {result_of_exam}""", reply_markup=tiles_keyboard)

        except Exception:
            ...

    @dispatcher.message_handler(lambda message: message.text == "Все верно. Продолжить")
    async def walk_on2(message):
        await select_city(message, count_of_exams, result_of_exam, profile)

    @dispatcher.message_handler(lambda message: message.text == "Неверно. Вести данные заново")
    async def reentry_data(message):
        await select_profile(message)


async def select_city(message, count_of_exams, result_of_exam, profile):
    # здесь пользователь выбирает город для поиска вуза
    tiles_keyboard = ReplyKeyboardMarkup(resize_keyboard=True)
    tiles_keyboard.add("Москва").add("Санкт-Петербург").add("Оренбург").add("Екатеринбург")

    await bot.send_message(message.chat.id, "Теперь выберите город из доступных ", reply_markup=tiles_keyboard)

    @dispatcher.message_handler(lambda message: message.text in CITIES)
    async def processing_of_choice_university(message, count_of_exams=count_of_exams, result_of_exam=result_of_exam):
        # расчет среднего балла результатов ЕГЭ
        med_result = result_of_exam / count_of_exams
        # ранг выставляется в соответствии с баллами ЕГЭ
        rank: str = "0"
        if 0 <= med_result <= 33:
            rank = "1"
        elif 34 <= med_result <= 55:
            rank = "2"
        elif 56 <= med_result <= 77:
            rank = "3"
        elif 78 <= med_result:
            rank = "4"

        text = str(universitys[profile][message.text][rank])

        await bot.send_message(message.chat.id, f"Я нашел для тебя следующий ВУЗ:\n{text}")


async def read_count_of_exams(message):
    try:
        return int(message.text)
    except TypeError:
        await bot.send_message(message.chat.id, "вы ввели неправильное число")
        await read_count_of_exams(message)
    except Exception:
        pass


async def read_result_of_exam(message):
    try:
        return int(message.text)
    except TypeError:
        await bot.send_message(message.chat.id, "вы ввели неправильное число")
        await read_result_of_exam(message)
    except Exception:
        pass


if __name__ == '__main__':
    executor.start_polling(dispatcher)


Cause exception while getting updates.
Traceback (most recent call last):
  File "d:\python\lib\site-packages\aiogram\dispatcher\dispatcher.py", line 381, in start_polling
    updates = await self.bot.get_updates(
  File "d:\python\lib\site-packages\aiogram\bot\bot.py", line 110, in get_updates
    result = await self.request(api.Methods.GET_UPDATES, payload)
  File "d:\python\lib\site-packages\aiogram\bot\base.py", line 236, in request
    return await api.make_request(await self.get_session(), self.server, self.__token, method, data, files,
  File "d:\python\lib\site-packages\aiogram\bot\api.py", line 140, in make_request
    return check_result(method, response.content_type, response.status, await response.text())
  File "d:\python\lib\site-packages\aiogram\bot\api.py", line 119, in check_result
    exceptions.ConflictError.detect(description)
  File "d:\python\lib\site-packages\aiogram\utils\exceptions.py", line 140, in detect
    raise err(cls.text or description)
aiogram.utils.ex

Goodbye!
Goodbye!
